In [1]:
import os
import sys
import urllib, io

import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [2]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.path.abspath(os.path.join(os.getcwd(),'..'))
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))
jefan_dir = os.path.join(analysis_dir,'jefan')
will_dir = os.path.join(analysis_dir,'will')

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [3]:
## set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## cocolab ip address

## have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['block_construction']
coll = db['silhouette']

## which iteration name should we use?
iterationName = 'pilot3'

## Sanity Checks

In [4]:
# Ensure one to one gameID and workerId 
# Should only happen if a repeat worker gets through

query = coll.find({"$and":[
                        {'workerId':{'$exists':True}},
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {"$or":[{'iterationName':'pilot3'},
                                {'iterationName':'pilot2'}]},
                        {'trialNum':0}]
                     })

df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))
#df_trial_end_full[['workerId','gameID']]


assert (np.mean(df_trial_end_full['workerId'].value_counts()) == np.mean(df_trial_end_full['gameID'].value_counts()))

In [5]:
# get ids of people with trial 15 data
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName': iterationName},
                        {'trialNum':15}]
                     })
complete_data_df = pd.DataFrame(query)
complete_data_ids = list(complete_data_df['workerId'])

In [6]:
complete_data_df['workerId'].nunique()

12

## Collect data from db and filter with sanity checks

In [8]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName':iterationName}]
                     })

df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))



# filter dataframe for complete datasets
df_trial_end_full_filtered = df_trial_end_full[df_trial_end_full.workerId.isin(complete_data_ids)]


# reduce to crucial information
df_trial_end_reduced_filtered = df_trial_end_full_filtered[['gameID','trialNum','phase','condition',
                                                            'eventType','score','normedScore','numBlocks',
                                                            'timeAbsolute','timeRelative','buildTime',
                                                            'currBonus','exploreResets','buildResets',
                                                            'allVertices','nPracticeAttempts','exploreStartTime',
                                                            'buildStartTime','buildFinishTime','targetName']]

df_for_analysis = df_trial_end_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])
df_for_analysis.head()


,gameID,trialNum,phase,condition,eventType,score,normedScore,numBlocks,timeAbsolute,timeRelative,buildTime,currBonus,exploreResets,buildResets,allVertices,nPracticeAttempts,exploreStartTime,buildStartTime,buildFinishTime,targetName
0,0335-390fd775-9a17-4e03-97de-fd31036a393d,0,build,mental,trial_end,0.00,0.412440,5,1.575400e+12,296387.370,60002,0.00,0,0,"[[{'x': 274.6545311714962, 'y': 592.6054374273...",None,1.575400e+12,1.575400e+12,1.575400e+12,hand_selected_004
1,0335-390fd775-9a17-4e03-97de-fd31036a393d,1,build,physical,trial_end,0.00,0.297015,3,1.575400e+12,392705.090,60003,0.00,0,0,"[[{'x': 232.00173028960907, 'y': 592.491661689...",None,1.575400e+12,1.575400e+12,1.575400e+12,hand_selected_008
2,0335-390fd775-9a17-4e03-97de-fd31036a393d,2,build,physical,trial_end,0.01,0.704637,7,1.575400e+12,489311.205,60002,0.01,0,0,"[[{'x': 277.97962409395717, 'y': 592.603205599...",None,1.575400e+12,1.575400e+12,1.575400e+12,hand_selected_011
5,0335-390fd775-9a17-4e03-97de-fd31036a393d,3,build,mental,trial_end,0.01,0.528537,6,1.575400e+12,585043.215,60003,0.00,0,0,"[[{'x': 245.94838550055294, 'y': 482.210460686...",None,1.575400e+12,1.575400e+12,1.575400e+12,hand_selected_009
10,0335-390fd775-9a17-4e03-97de-fd31036a393d,4,build,physical,trial_end,0.01,0.230413,7,1.575400e+12,683986.240,60022,0.00,0,0,"[[{'x': 307.0867862202134, 'y': 647.5002866143...",None,1.575400e+12,1.575400e+12,1.575400e+12,hand_selected_016


In [ ]:
## save out to csv dir, where all the csv's go to live
out_path = os.path.join(csv_dir,'block_silhouette_{}.csv'.format(iterationName))
df_for_analysis.to_csv(out_path)

## Settled Block Data

In [63]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'settled'},
                        {'iterationName':iterationName}]
                     })

df_settled_full = pd.DataFrame(list(query.sort('timeAbsolute')))


# filter dataframe for complete datasets
df_settled_full_filtered = df_settled_full[df_settled_full.workerId.isin(complete_data_ids)]

df_settled_full_filtered
# reduce to crucial information

,F1Score,_id,aID,allBlockBodyProperties,allBlockDims,allVertices,blockBodyProperties,blockCenterX,blockCenterY,blockDimUnits,...,targetName,timeAbsolute,timeRelative,trialNum,version,vertices,workerId,worldHeightUnits,worldScale,worldWidthUnits
0,0.000000,5de6b0b2a21b6d02ba436012,3HQUKB7LNGSSPILZ6EGP0GRNHWZHHA,"[{'id': 31, 'angle': 2.516542152836914e-19, 'p...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 274.75000000000006, 'y': 592.478571428...","{'id': 31, 'type': 'body', 'label': 'Rectangle...",384.750000,647.478571,"[4, 2]",...,hand_selected_004,1.575400e+12,252952.170,0,81,"[{'x': -247.50000000000017, 'y': 702.428571428...",A17K1CHOI773VZ,8,2.2,8
1,0.000000,5de6b0bba21b6d02ba436015,3HQUKB7LNGSSPILZ6EGP0GRNHWZHHA,"[{'id': 31, 'angle': -1.447434604736027e-06, '...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 274.7497028650838, 'y': 592.4968162760...","{'id': 33, 'type': 'body', 'label': 'Rectangle...",386.999542,537.547061,"[4, 2]",...,hand_selected_004,1.575400e+12,261745.680,0,81,"[{'x': -247.50000000000017, 'y': 702.428571428...",A17K1CHOI773VZ,8,2.2,8
2,0.000000,5de6b0c7a21b6d02ba43601b,3HQUKB7LNGSSPILZ6EGP0GRNHWZHHA,"[{'id': 31, 'angle': 2.3366267228919768e-05, '...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 274.7339101765754, 'y': 592.5879721237...","{'id': 35, 'type': 'body', 'label': 'Rectangle...",468.026584,427.854436,"[4, 2]",...,hand_selected_004,1.575400e+12,273395.275,0,81,"[{'x': -247.50000000000017, 'y': 702.428571428...",A17K1CHOI773VZ,8,2.2,8
3,0.000000,5de6b0d9a21b6d02ba436023,3HQUKB7LNGSSPILZ6EGP0GRNHWZHHA,"[{'id': 31, 'angle': 2.4020500673359973e-05, '...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 274.72748765989786, 'y': 592.587851807...","{'id': 37, 'type': 'body', 'label': 'Rectangle...",560.250000,647.478571,"[2, 2]",...,hand_selected_004,1.575400e+12,291718.450,0,81,"[{'x': -247.50000000000017, 'y': 702.428571428...",A17K1CHOI773VZ,8,2.2,8
4,0.507370,5de6b0dea21b6d02ba436027,3HQUKB7LNGSSPILZ6EGP0GRNHWZHHA,"[{'id': 31, 'angle': 0.00012605003680354112, '...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 274.6545311714962, 'y': 592.6054374273...","{'id': 46, 'type': 'body', 'label': 'Rectangle...",870.338644,589.856989,"[4, 2]",...,hand_selected_004,1.575400e+12,296308.950,0,81,"[{'x': -247.50000000000017, 'y': 702.428571428...",A17K1CHOI773VZ,8,2.2,8
5,0.000000,5de6b0f8a21b6d02ba43603c,3HQUKB7LNGSSPILZ6EGP0GRNHWZHHA,"[{'id': 53, 'angle': 0, 'position': {'x': 319....","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 264.5, 'y': 482.47857142857384}, {'x':...","{'id': 53, 'type': 'body', 'label': 'Rectangle...",319.500000,592.478571,"[2, 4]",...,hand_selected_008,1.575400e+12,322714.150,1,81,"[{'x': -247.50000000000017, 'y': 702.428571428...",A17K1CHOI773VZ,8,2.2,8
6,0.000000,5de6b11aa21b6d02ba436062,3HQUKB7LNGSSPILZ6EGP0GRNHWZHHA,"[{'id': 62, 'angle': 2.2259515440461917e-19, '...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 232.00000000000006, 'y': 592.478571428...","{'id': 62, 'type': 'body', 'label': 'Rectangle...",342.000000,647.478571,"[4, 2]",...,hand_selected_008,1.575400e+12,356517.760,1,81,"[{'x': -247.50000000000017, 'y': 702.428571428...",A17K1CHOI773VZ,8,2.2,8
7,0.000000,5de6b138a21b6d02ba436077,3HQUKB7LNGSSPILZ6EGP0GRNHWZHHA,"[{'id': 62, 'angle': 2.392389314683121e-19, 'p...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 232.00000000000006, 'y': 592.478571428...","{'id': 64, 'type': 'body', 'label': 'Rectangle...",171.000000,592.478571,"[2, 4]",...,hand_selected_008,1.575400e+12,386743.110,1,81,"[{'x': -247.50000000000017, 'y': 702.428571428...",A17K1CHOI773VZ,8,2.2,8
8,0.413737,5de6b13ea21b6d02ba43607e,3HQUKB7LNGSSPILZ6EGP0GRNHWZHHA,"[{'id': 62, 'angle': 2.342901592778402e-05, 'p...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 232.00173028960907, 'y': 592.491661689...","{'id': 66, 'type': 'body', 'label': 'Rectangle...",404.961348,482.523354,"[2, 4]",...,hand_selected_008,1.575400e+12,392636.630,1,81,"[{'x': -247.5000000000

In [75]:

df_settled_reduced_filtered = df_settled_full_filtered[['gameID','trialNum','phase','condition',
                                                            'eventType','numBlocks', 'timeAbsolute','timeRelative',
                                                            'normedScore','currBonus','score','incrementalScore','normedIncrementalScore',
                                                            'currBonus','allVertices','targetName'
                                                           ]]

df_settled_reduced_filtered = df_settled_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])

buildstart = df_for_analysis[['gameID','trialNum','buildStartTime','exploreStartTime']]
df_settled_reduced_filtered = df_settled_reduced_filtered.merge(buildstart, on=['gameID', 'trialNum'], how='left')
df_settled_reduced_filtered['timePlaced'] = df_settled_reduced_filtered['timeAbsolute'] - df_settled_reduced_filtered['buildStartTime']
df_settled_reduced_filtered.loc[(df_settled_reduced_filtered.timePlaced < 0),'timePlaced'] = df_settled_reduced_filtered[df_settled_reduced_filtered.timePlaced < 0]['timeAbsolute'] - df_settled_reduced_filtered[df_settled_reduced_filtered.timePlaced < 0]['exploreStartTime']


In [65]:
df_settled_reduced_filtered

,gameID,trialNum,phase,condition,eventType,numBlocks,timeAbsolute,timeRelative,normedScore,currBonus,score,incrementalScore,normedIncrementalScore,currBonus,allVertices,targetName,buildStartTime,timePlaced
0,0335-390fd775-9a17-4e03-97de-fd31036a393d,0,build,mental,settled,1,1.575400e+12,252952.170,0.000000,0.00,0.00,0.495610,0.398413,0.00,"[[{'x': 274.75000000000006, 'y': 592.478571428...",hand_selected_004,1.575400e+12,16696.0
1,0335-390fd775-9a17-4e03-97de-fd31036a393d,0,build,mental,settled,2,1.575400e+12,261745.680,0.000000,0.00,0.00,0.634016,0.563491,0.00,"[[{'x': 274.7497028650838, 'y': 592.4968162760...",hand_selected_004,1.575400e+12,25489.0
2,0335-390fd775-9a17-4e03-97de-fd31036a393d,0,build,mental,settled,3,1.575400e+12,273395.275,0.000000,0.00,0.00,0.708543,0.652379,0.00,"[[{'x': 274.7339101765754, 'y': 592.5879721237...",hand_selected_004,1.575400e+12,37139.0
3,0335-390fd775-9a17-4e03-97de-fd31036a393d,0,build,mental,settled,4,1.575400e+12,291718.450,0.000000,0.00,0.00,0.757506,0.710777,0.00,"[[{'x': 274.72748765989786, 'y': 592.587851807...",hand_selected_004,1.575400e+12,55462.0
4,0335-390fd775-9a17-4e03-97de-fd31036a393d,0,build,mental,settled,5,1.575400e+12,296308.950,0.412440,0.00,0.00,0.507370,0.412440,0.00,"[[{'x': 274.6545311714962, 'y': 592.6054374273...",hand_selected_004,1.575400e+12,60053.0
5,0335-390fd775-9a17-4e03-97de-fd31036a393d,1,explore,physical,settled,1,1.575400e+12,322714.150,0.000000,0.00,0.00,0.386341,0.264165,0.00,"[[{'x': 264.5, 'y': 482.47857142857384}, {'x':...",hand_selected_008,1.575400e+12,-9875.0
6,0335-390fd775-9a17-4e03-97de-fd31036a393d,1,build,physical,settled,1,1.575400e+12,356517.760,0.000000,0.00,0.00,0.242786,0.092029,0.00,"[[{'x': 232.00000000000006, 'y': 592.478571428...",hand_selected_008,1.575400e+12,23928.0
7,0335-390fd775-9a17-4e03-97de-fd31036a393d,1,build,physical,settled,2,1.575400e+12,386743.110,0.000000,0.00,0.00,0.400725,0.281413,0.00,"[[{'x': 232.00000000000006, 'y': 592.478571428...",hand_selected_008,1.575400e+12,54154.0
8,0335-390fd775-9a17-4e03-97de-fd31036a393d,1,build,physical,settled,3,1.575400e+12,392636.630,0.297015,0.00,0.00,0.413737,0.297015,0.00,"[[{'x': 232.00173028960907, 'y': 592.491661689...",hand_selected_008,1.575400e+12,60048.0
9,0335-390fd775-9a17-4e03-97de-fd31036a393d,2,explore,physical,settled,1,1.575400e+12,408077.615,0.000000,0.00,0.00,0.418605,0.307542,0.00,"[[{'x': 265.75000000000006, 'y': 592.478571428...",hand_selected_011,1.575400e+12,-21097.0


In [79]:
## save out to csv dir, where all the csv's go to live
out_path = os.path.join(csv_dir,'block_silhouette_settled_{}.csv'.format(iterationName))
df_settled_reduced_filtered.to_csv(out_path)

In [67]:
## which iteration name should we use?
iterationName = 'pilot3'

## load in dataframe
data_path = os.path.join(csv_dir,'block_silhouette_settled_{}.csv'.format(iterationName))
df = pd.read_csv(data_path)
df

,Unnamed: 0,gameID,trialNum,phase,condition,eventType,numBlocks,timeAbsolute,timeRelative,normedScore,currBonus,score,incrementalScore,normedIncrementalScore,currBonus.1,allVertices,targetName,buildStartTime,timePlaced
0,0,0335-390fd775-9a17-4e03-97de-fd31036a393d,0,build,mental,settled,1,1.575400e+12,252952.170,0.000000,0.00,0.00,0.495610,0.398413,0.00,"[[{'x': 274.75000000000006, 'y': 592.478571428...",hand_selected_004,1.575400e+12,16696.0
1,1,0335-390fd775-9a17-4e03-97de-fd31036a393d,0,build,mental,settled,2,1.575400e+12,261745.680,0.000000,0.00,0.00,0.634016,0.563491,0.00,"[[{'x': 274.7497028650838, 'y': 592.4968162760...",hand_selected_004,1.575400e+12,25489.0
2,2,0335-390fd775-9a17-4e03-97de-fd31036a393d,0,build,mental,settled,3,1.575400e+12,273395.275,0.000000,0.00,0.00,0.708543,0.652379,0.00,"[[{'x': 274.7339101765754, 'y': 592.5879721237...",hand_selected_004,1.575400e+12,37139.0
3,3,0335-390fd775-9a17-4e03-97de-fd31036a393d,0,build,mental,settled,4,1.575400e+12,291718.450,0.000000,0.00,0.00,0.757506,0.710777,0.00,"[[{'x': 274.72748765989786, 'y': 592.587851807...",hand_selected_004,1.575400e+12,55462.0
4,4,0335-390fd775-9a17-4e03-97de-fd31036a393d,0,build,mental,settled,5,1.575400e+12,296308.950,0.412440,0.00,0.00,0.507370,0.412440,0.00,"[[{'x': 274.6545311714962, 'y': 592.6054374273...",hand_selected_004,1.575400e+12,60053.0
5,5,0335-390fd775-9a17-4e03-97de-fd31036a393d,1,explore,physical,settled,1,1.575400e+12,322714.150,0.000000,0.00,0.00,0.386341,0.264165,0.00,"[[{'x': 264.5, 'y': 482.47857142857384}, {'x':...",hand_selected_008,1.575400e+12,-9875.0
6,6,0335-390fd775-9a17-4e03-97de-fd31036a393d,1,build,physical,settled,1,1.575400e+12,356517.760,0.000000,0.00,0.00,0.242786,0.092029,0.00,"[[{'x': 232.00000000000006, 'y': 592.478571428...",hand_selected_008,1.575400e+12,23928.0
7,7,0335-390fd775-9a17-4e03-97de-fd31036a393d,1,build,physical,settled,2,1.575400e+12,386743.110,0.000000,0.00,0.00,0.400725,0.281413,0.00,"[[{'x': 232.00000000000006, 'y': 592.478571428...",hand_selected_008,1.575400e+12,54154.0
8,8,0335-390fd775-9a17-4e03-97de-fd31036a393d,1,build,physical,settled,3,1.575400e+12,392636.630,0.297015,0.00,0.00,0.413737,0.297015,0.00,"[[{'x': 232.00173028960907, 'y': 592.491661689...",hand_selected_008,1.575400e+12,60048.0
9,9,0335-390fd775-9a17-4e03-97de-fd31036a393d,2,explore,physical,settled,1,1.575400e+12,408077.615,0.000000,0.00,0.00,0.418605,0.307542,0.00,"[[{'x': 265.75000000000006, 'y': 592.478571428...",hand_selected_011,1.575400e+12,-21097.0
